In [1]:
import pm4py
import sys
sys.path.append('../../TaskExecutionTimeMining/')
from event_log_transformer import *

In [2]:
file_path = '../../../data/BPI Challenge 2017.xes'
event_log = pm4py.read_xes(file_path)

parsing log, completed traces ::   0%|          | 0/31509 [00:00<?, ?it/s]

In [3]:
event_log

,Action,org:resource,concept:name,EventOrigin,EventID,lifecycle:transition,time:timestamp,case:LoanGoal,case:ApplicationType,case:concept:name,case:RequestedAmount,FirstWithdrawalAmount,NumberOfTerms,Accepted,MonthlyCost,Selected,CreditScore,OfferedAmount,OfferID
0,Created,User_1,A_Create Application,Application,Application_652823628,complete,2016-01-01 09:51:15.304000+00:00,Existing loan takeover,New credit,Application_652823628,20000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,statechange,User_1,A_Submitted,Application,ApplState_1582051990,complete,2016-01-01 09:51:15.352000+00:00,Existing loan takeover,New credit,Application_652823628,20000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Created,User_1,W_Handle leads,Workflow,Workitem_1298499574,schedule,2016-01-01 09:51:15.774000+00:00,Existing loan takeover,New credit,Application_652823628,20000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Deleted,User_1,W_Handle leads,Workflow,Workitem_1673366067,withdraw,2016-01-01 09:52:36.392000+00:00,Existing loan takeover,New credit,Application_652823628,20000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Created,User_1,W_Complete application,Workflow,Workitem_1493664571,schedule,2016-01-01 09:52:36.403000+00:00,Existing loan takeover,New credit,Application_652823628,20000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1202262,Deleted,User_1,W_Call after offers,Workflow,Workitem_1817549786,ate_abort,2017-01-06 06:33:02.212000+00:00,Home improvement,New credit,Application_1350494635,20000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1202263,Created,User_1,W_Call after offers,Workflow,Workitem_363876066,schedule,2017-01-06 06:33:02.221000+00:00,Home improvement,New credit,Application_1350494635,20000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1202264,statechange,User_28,A_Cancelled,Application,ApplState_1869071797,complete,2017-01-16 09:51:21.114000+00:00,Home improvement,New credit,Application_1350494635,20000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1202265,statechange,User_28,O_Cancelled,Offer,OfferState_420066181,complete,2017-01-16 09:51:21.139000+00:00,Home improvement,New credit,Application_1350494635,20000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Offer_1580299144


In [4]:
start_end_event_log = TransformEventLog.start_end_event_log(event_log, start_name = 'start', complete_name='complete')

/home/michel/Documents/TaskExecutionTimeMining/src/notebooks/BPIC_2017/../../TaskExecutionTimeMining/event_log_transformer.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  start_end_event_log.loc[:, 'duration'] = start_end_event_log[timestamp_name + '_complete'] - start_end_event_log[timestamp_name + '_start']
/home/michel/Documents/TaskExecutionTimeMining/src/notebooks/BPIC_2017/../../TaskExecutionTimeMining/event_log_transformer.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  start_end_event_log.

In [5]:
# Seconds in day
start_end_event_log = TransformEventLog.seconds_in_day(start_end_event_log, 'time:timestamp_start')

In [6]:
start_end_event_log

,Action_start,org:resource,concept:name,EventOrigin_start,EventID_start,lifecycle:transition_start,time:timestamp_start,case:LoanGoal_start,case:ApplicationType_start,case:concept:name,...,NumberOfTerms_complete,Accepted_complete,MonthlyCost_complete,Selected_complete,CreditScore_complete,OfferedAmount_complete,OfferID_complete,duration,duration_seconds,seconds_in_day
158,Obtained,User_19,W_Complete application,Workflow,Workitem_62767814,start,2016-01-02 10:55:03.169000+00:00,Home improvement,New credit,Application_428409768,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0 days 00:11:50.122000,710,39303
246,Obtained,User_17,W_Handle leads,Workflow,Workitem_1237205341,start,2016-01-02 09:05:02.658000+00:00,Car,New credit,Application_1746793196,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0 days 00:01:01.543000,61,32702
507,Obtained,User_19,W_Complete application,Workflow,Workitem_1987035323,start,2016-01-02 11:32:57.796000+00:00,Home improvement,New credit,Application_828200680,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0 days 00:10:58.463000,658,41577
542,Obtained,User_17,W_Complete application,Workflow,Workitem_1299269231,start,2016-01-02 11:22:55.931000+00:00,Existing loan takeover,New credit,Application_1085880569,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0 days 00:22:15.248000,1335,40975
824,Obtained,User_19,W_Complete application,Workflow,Workitem_488414588,start,2016-01-02 14:26:27.976000+00:00,Car,New credit,Application_619403287,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0 days 00:18:26.634000,1106,51987
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2873477,Obtained,User_37,W_Complete application,Workflow,Workitem_690805293,start,2016-12-31 13:35:46.637000+00:00,Car,New credit,Application_1280325005,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0 days 00:04:37.821000,277,48946
2873733,Obtained,User_52,W_Complete application,Workflow,Workitem_1759805453,start,2016-12-31 14:40:38.702000+00:00,Unknown,Limit raise,Application_770397679,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0 days 00:04:48.673000,288,52838
2873793,Obtained,User_66,W_Handle leads,Workflow,Workitem_210386077,start,2017-01-02 07:01:22.462000+00:00,Car,New credit,Application_117342811,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0 days 00:00:44.325000,44,25282
2873839,Obtained,User_53,W_Validate application,Workflow,Workitem_1661538584,start,2017-01-11 14:23:08.405000+00:00,Car,New credit,Application_117342811,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0 days 00:03:31.124000,211,51788


In [7]:
# Resource counts

resource_count_event_log = TransformEventLog.value_count_per_case(start_end_event_log, 'org:resource',
                                                                  timestamp_name = 'time:timestamp_start',
                                                                 lifecycle_col_name = 'lifecycle:transition_start' )

/home/michel/Documents/TaskExecutionTimeMining/src/notebooks/BPIC_2017/../../TaskExecutionTimeMining/event_log_transformer.py:54: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  value_count_event_log = value_count_event_log.fillna(0)


In [8]:
resource_count_event_log

,Action_start,org:resource,concept:name,EventOrigin_start,EventID_start,lifecycle:transition_start,time:timestamp_start,case:LoanGoal_start,case:ApplicationType_start,case:concept:name,...,User_90,User_91,User_92,User_93,User_94,User_95,User_96,User_97,User_98,User_99
0,Obtained,User_19,W_Complete application,Workflow,Workitem_62767814,start,2016-01-02 10:55:03.169000+00:00,Home improvement,New credit,Application_428409768,...,0,0,0,0,0,0,0,0,0,0
1,Obtained,User_17,W_Handle leads,Workflow,Workitem_1237205341,start,2016-01-02 09:05:02.658000+00:00,Car,New credit,Application_1746793196,...,0,0,0,0,0,0,0,0,0,0
2,Obtained,User_19,W_Complete application,Workflow,Workitem_1987035323,start,2016-01-02 11:32:57.796000+00:00,Home improvement,New credit,Application_828200680,...,0,0,0,0,0,0,0,0,0,0
3,Obtained,User_17,W_Complete application,Workflow,Workitem_1299269231,start,2016-01-02 11:22:55.931000+00:00,Existing loan takeover,New credit,Application_1085880569,...,0,0,0,0,0,0,0,0,0,0
4,Obtained,User_19,W_Complete application,Workflow,Workitem_488414588,start,2016-01-02 14:26:27.976000+00:00,Car,New credit,Application_619403287,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22166,Obtained,User_37,W_Complete application,Workflow,Workitem_690805293,start,2016-12-31 13:35:46.637000+00:00,Car,New credit,Application_1280325005,...,0,0,0,0,0,0,0,0,0,0
22167,Obtained,User_52,W_Complete application,Workflow,Workitem_1759805453,start,2016-12-31 14:40:38.702000+00:00,Unknown,Limit raise,Application_770397679,...,0,0,0,0,0,0,0,0,0,0
22168,Obtained,User_66,W_Handle leads,Workflow,Workitem_210386077,start,2017-01-02 07:01:22.462000+00:00,Car,New credit,Application_117342811,...,0,0,0,0,0,0,0,0,0,0
22169,Obtained,User_53,W_Validate application,Workflow,Workitem_1661538584,start,2017-01-11 14:23:08.405000+00:00,Car,New credit,Application_117342811,...,0,0,0,0,0,0,0,0,0,0


In [9]:
activity_count_event_log = TransformEventLog.value_count_per_case(resource_count_event_log, 'concept:name',
                                                                  timestamp_name = 'time:timestamp_start',
                                                                 lifecycle_col_name = 'lifecycle:transition_start' )

In [10]:
activity_count_event_log

,Action_start,org:resource,concept:name,EventOrigin_start,EventID_start,lifecycle:transition_start,time:timestamp_start,case:LoanGoal_start,case:ApplicationType_start,case:concept:name,...,User_96,User_97,User_98,User_99,W_Assess potential fraud,W_Call after offers,W_Call incomplete files,W_Complete application,W_Handle leads,W_Validate application
0,Obtained,User_19,W_Complete application,Workflow,Workitem_62767814,start,2016-01-02 10:55:03.169000+00:00,Home improvement,New credit,Application_428409768,...,0,0,0,0,0,0,0,1,0,0
1,Obtained,User_17,W_Handle leads,Workflow,Workitem_1237205341,start,2016-01-02 09:05:02.658000+00:00,Car,New credit,Application_1746793196,...,0,0,0,0,0,0,0,0,1,0
2,Obtained,User_19,W_Complete application,Workflow,Workitem_1987035323,start,2016-01-02 11:32:57.796000+00:00,Home improvement,New credit,Application_828200680,...,0,0,0,0,0,0,0,1,0,0
3,Obtained,User_17,W_Complete application,Workflow,Workitem_1299269231,start,2016-01-02 11:22:55.931000+00:00,Existing loan takeover,New credit,Application_1085880569,...,0,0,0,0,0,0,0,1,0,0
4,Obtained,User_19,W_Complete application,Workflow,Workitem_488414588,start,2016-01-02 14:26:27.976000+00:00,Car,New credit,Application_619403287,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22166,Obtained,User_37,W_Complete application,Workflow,Workitem_690805293,start,2016-12-31 13:35:46.637000+00:00,Car,New credit,Application_1280325005,...,0,0,0,0,0,0,0,1,0,0
22167,Obtained,User_52,W_Complete application,Workflow,Workitem_1759805453,start,2016-12-31 14:40:38.702000+00:00,Unknown,Limit raise,Application_770397679,...,0,0,0,0,0,0,0,1,0,0
22168,Obtained,User_66,W_Handle leads,Workflow,Workitem_210386077,start,2017-01-02 07:01:22.462000+00:00,Car,New credit,Application_117342811,...,0,0,0,0,0,0,0,0,1,0
22169,Obtained,User_53,W_Validate application,Workflow,Workitem_1661538584,start,2017-01-11 14:23:08.405000+00:00,Car,New credit,Application_117342811,...,0,0,0,0,0,0,0,0,1,1


In [11]:
activity_count_event_log.to_csv('../transformed_event_logs/BPIC_2017_start_end.csv')
activity_count_event_log.to_pickle('../transformed_event_logs/BPIC_2017_start_end.pickle')